## Validacion de modelos

In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.metrics import Precision, Recall
from keras import backend as K
# Importar las bibliotecas necesarias
import os
import numpy as np
from tensorflow.keras.preprocessing import image



In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def train_val_generators(train_dir, val_dir):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,  # Added zoom range
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
    )

    val_datagen = ImageDataGenerator(rescale=1./255.)

    train_generator = train_datagen.flow_from_directory(
        directory=train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )

    val_generator = val_datagen.flow_from_directory(
        directory=val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )

    return train_generator, val_generator


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

# Directorio base después de montar el drive
base_path = '/content/drive/My Drive/DataSet'

# Listar archivos en el directorio base
files = os.listdir(base_path)

path_root = '/content/drive/My Drive/DataSet'

valid_dir = os.path.join(path_root, 'val')
train_dir = os.path.join(path_root, 'train')
test_dir = os.path.join(path_root, 'test')

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set IMG_SIZE and BATCH_SIZE
IMG_SIZE = (512, 512)
BATCH_SIZE = 8
NUM_CLASES = 4



In [6]:
train_generator, val_generator = train_val_generators(train_dir, valid_dir)   # create image generators to feed images to the model


Found 236 images belonging to 4 classes.
Found 81 images belonging to 4 classes.


In [7]:
# Obtener el diccionario de asignación de clases a índices
class_indices = train_generator.class_indices

# Imprimir el diccionario
print(class_indices)


{'Benign': 0, 'InSitu': 1, 'Invasive': 2, 'Normal': 3}


In [8]:
def load_image(img_path):
    img = image.load_img(img_path, target_size=(IMG_SIZE[0], IMG_SIZE[1]))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalizar los píxeles
    return img_array


In [9]:
#Para poder incluir el valor f1 en el entrenamiento del modelo creamos las siguientes
#funciones para añadirlo como una función siguiendo la referencia del sigueinte enlace
#https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model

from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))



In [10]:


# Ruta donde se guardó el modelo
model_path = '/content/drive/My Drive/SavedModels/xception_model_adam.h5'

# Cargar el modelo
modelo_entrenado = load_model(model_path, custom_objects={'precision_m': precision_m, 'f1_m': f1_m,'recall_m': recall_m})




In [11]:

# Directorio de la carpeta de prueba
test_path = '/content/drive/My Drive/Test'


In [12]:
from PIL import Image
import numpy as np

# Ruta a la carpeta que contiene las imágenes
ruta_imagenes =  test_path

# Lista para almacenar las imágenes y sus nombres
imagenes = []
nombres = []

# Recorre los archivos en la carpeta
for archivo in os.listdir(ruta_imagenes):
    # Ignora archivos que no son imágenes
    if archivo.endswith(('.tiff', '.tif')):
        ruta_imagen = os.path.join(ruta_imagenes, archivo)
        # Carga la imagen y redimensiona si es necesario
        img = image.load_img(ruta_imagen, target_size=(IMG_SIZE[0], IMG_SIZE[1]))
        img_array = image.img_to_array(img)
        img_array /= 255.0  # Normalizar los píxeles
        nombres.append(archivo)
        imagenes.append(img_array)




In [13]:
imagenes[1].shape

(512, 512, 3)

In [14]:
tensor_de_imagenes = np.stack(imagenes)
tensor_de_imagenes.shape

(100, 512, 512, 3)

In [15]:
predicciones = modelo_entrenado.predict(tensor_de_imagenes)


4/4 [==============================] - 20s 1s/step


In [16]:
clases_predichas = np.argmax(predicciones, axis=1)
clases_predichas

array([2, 3, 2, 2, 2, 0, 0, 3, 3, 1, 1, 1, 2, 0, 3, 3, 1, 2, 3, 2, 1, 0,
       1, 3, 1, 3, 2, 3, 0, 2, 0, 3, 3, 3, 1, 2, 2, 0, 2, 1, 2, 2, 1, 3,
       1, 2, 2, 1, 1, 3, 1, 0, 2, 3, 2, 3, 2, 3, 1, 0, 2, 2, 3, 3, 2, 3,
       1, 2, 2, 1, 3, 0, 1, 0, 3, 1, 0, 1, 2, 3, 2, 3, 2, 3, 2, 2, 1, 3,
       3, 1, 1, 1, 3, 0, 3, 0, 1, 0, 1, 1])

In [17]:
# Diccionario de asignación de índices a nombres de clases
indice_a_clase = {0: 'Benign', 1: 'InSitu', 2: 'Invasive', 3: 'Normal'}

# Mapear las clases predichas de índices a nombres de clases
clases_predichas_nombres = [indice_a_clase[indice] for indice in clases_predichas]

# Imprimir las clases predichas en nombres
print(clases_predichas_nombres)


['Invasive', 'Normal', 'Invasive', 'Invasive', 'Invasive', 'Benign', 'Benign', 'Normal', 'Normal', 'InSitu', 'InSitu', 'InSitu', 'Invasive', 'Benign', 'Normal', 'Normal', 'InSitu', 'Invasive', 'Normal', 'Invasive', 'InSitu', 'Benign', 'InSitu', 'Normal', 'InSitu', 'Normal', 'Invasive', 'Normal', 'Benign', 'Invasive', 'Benign', 'Normal', 'Normal', 'Normal', 'InSitu', 'Invasive', 'Invasive', 'Benign', 'Invasive', 'InSitu', 'Invasive', 'Invasive', 'InSitu', 'Normal', 'InSitu', 'Invasive', 'Invasive', 'InSitu', 'InSitu', 'Normal', 'InSitu', 'Benign', 'Invasive', 'Normal', 'Invasive', 'Normal', 'Invasive', 'Normal', 'InSitu', 'Benign', 'Invasive', 'Invasive', 'Normal', 'Normal', 'Invasive', 'Normal', 'InSitu', 'Invasive', 'Invasive', 'InSitu', 'Normal', 'Benign', 'InSitu', 'Benign', 'Normal', 'InSitu', 'Benign', 'InSitu', 'Invasive', 'Normal', 'Invasive', 'Normal', 'Invasive', 'Normal', 'Invasive', 'Invasive', 'InSitu', 'Normal', 'Normal', 'InSitu', 'InSitu', 'InSitu', 'Normal', 'Benign', '

In [18]:


# Crear un DataFrame con las columnas requeridas
df_resultados = pd.DataFrame({
    'Nombre de la Imagen': nombres,
    'Etiqueta Real': clases_predichas,
    'Clase Predicha': clases_predichas_nombres
})
df_resultados



,Nombre de la Imagen,Etiqueta Real,Clase Predicha
0,test27.tif,2,Invasive
1,test14.tif,3,Normal
2,test15.tif,2,Invasive
3,test18.tif,2,Invasive
4,test1.tif,2,Invasive
...,...,...,...
95,test90.tif,0,Benign
96,test76.tif,1,InSitu
97,test59.tif,0,Benign
98,test91.tif,1,InSitu


In [19]:
valores_distintos = df_resultados[['Etiqueta Real', 'Clase Predicha']].drop_duplicates()

print(valores_distintos)

   Etiqueta Real Clase Predicha
0              2       Invasive
1              3         Normal
5              0         Benign
9              1         InSitu


In [21]:
# Muestra el DataFrame

output_dir = '/content/drive/My Drive/Results'
csv_filename = os.path.join(output_dir, f'xception_model_adam.csv')
df_resultados.to_csv(csv_filename, index=False)
